# Tutorial: Generating Steric Descriptors for Alcohol Molecules

Author: Hideya Tanaka

Reviewer: Tomoyuki Miyao

---

This tutorial demonstrates the step-by-step generation of steric descriptors for a dataset of primary and secondary alcohols, based on MORFEUS steric analyses implemented through two separate modules:

- **morfeus_vbur_sterimol_alcohol.py (Module 1)**
- **morfeus_vbur_quadrant_octant_alcohol.py (Module 2)**

### Table of Contents

1. **Data Curation**
2. **Conformer Search and Quantum Chemical Calculations**
3. **Generating Steric Descriptors with Module 1**
4. **Generating Steric Descriptors with Module 2**
5. **Generating Steric Profiles**

---

## 1. Data Curation

Based on expert knowledge, we selected molecules with diverse structures that contain only a single primary or secondary alcohol group.
- Since the target reaction is alcohol oxidation, we constructed a dataset consisting of primary and secondary alcohols.
- If a molecule contains more than one potentially reactive alcohol site, the reaction becomes more complex, and the interpretability of the steric descriptors is significantly diminished. Therefore, only molecules containing a single primary or secondary alcohol site were included in the dataset.

## 2. Conformer Search and Quantum Chemical Calculations

We conducted a thorough conformer search workflow that combines [RDKit](https://www.rdkit.org/) and [xTB](https://github.com/grimme-lab/xtb), followed by a final geometry optimization at the M06-2X/def2-SVP level of theory in [Gaussian 16 Rev. C.01](https://gaussian.com/relnotes/):
1. **RDKit-based Conformer Generation**

    - We used [MORFEUS](https://github.com/digital-chemistry-laboratory/morfeus) conformer tools, which incorporate RDKit, to generate up to 200 conformers per molecule with the following parameters:
        > ce = ConformerEnsemble.from_rdkit(smiles, n_confs=200, optimize='MMFF94', random_seed=42, rmsd_thres=0)

        > ce.prune_rmsd(thres=1)

After pruning conformers with an RMSD threshold of 1 Å, we selected up to 50 of the lowest-energy conformers per molecule in ascending order of energy (based on RDKit’s MMFF94 energy).

2. **xTB Optimizations**

    - The selected conformers were optimized in xTB using the following command:
        > xtb {file_path} --ohess --chrg 0 --json

    - All optimized structures were confirmed to be local minima by verifying the absence of imaginary frequencies.
3. **Gaussian Calculations**
    - The Gaussian calculations were performed using the following keywords in the route section:
        >  \#p M062X/Def2SVP opt freq=noraman

    - Among the xTB-optimized geometries, the lowest-energy conformer was chosen as the starting geometry for a full optimization at the M06-2X/def2-SVP level of theory using Gaussian 16 Rev. C.01.
    - All optimized structures were confirmed to be local minima by verifying the absence of imaginary frequencies.
    - All final log files were parsed via [cclib](https://github.com/cclib/cclib) to generate xyz files (geometries).
    - The resulting xyz files were saved into the directory: `steric-descriptors-alcohol/data/input_data/gaussian_optimized_xyz`
    - The sdf file corresponding to the xyz files was also saved: `steric-descriptors-alcohol/data/input_data/gaussian_optimized_mols.sdf`
    - The CSV file containing the dataset (including thermodynamic and orbital data) is `steric-descriptors-alcohol/data/input_data/input_dataset_alcohol_oxidation.csv`. Each conformer was assigned a `confid`, which is a sequential number starting from 1, ordered according to increasing RDKit MMFF94 energy.

## 3. Generating Steric Descriptors with Module 1

The following sections describe the generation of MORFEUS-based steric descriptors for our curated dataset using two Python modules: Module 1 and Module 2.   

By specifying different combinations of `center_atom` and `attached_atom` in a function provided by Module 1, we obtained a comprehensive set of steric descriptors focusing on the reactive OH site of alcohols.
We also performed a radius scan for each atom specification pattern.
This comprehensive generation of steric descriptors not only enables the identification of optimal descriptors for the reaction but also allows for a fair evaluation of how differences in the specified atoms used for steric descriptor calculation affect the performance of the yield prediction model.
In all cases, we assigned either the OH moiety of the primary or secondary alcohol, the carbon atom bonded to that OH, or the hydrogen atom attached to that carbon as the `center_atom` or `attached_atom`.  
For usage details, please refer to each respective module.

The script `morfeus_vbur_sterimol_alcohol.py` (Module 1) provides the function `process_rows_for_morfeus`, which calculates:
- Percent buried volume (%Vbur), both including and excluding hydrogen atoms.
- Sterimol parameters (B1, B5, L) in both “standard” and “buried” forms.

This module appends the newly generated descriptor columns directly to the input CSV file. Therefore, we ran it four times consecutively, each time specifying different arguments for `center_atom` and `attached_atom`.


In [ ]:
import os
import sys

fd = os.getcwd()
parent_dir = os.path.dirname(fd)
sys.path.append(parent_dir)

from src.morfeus_vbur_sterimol_alcohol import process_rows_for_morfeus

input_dir_path = f'{parent_dir}/data/input_data'
xyz_dir_path_input = f'{input_dir_path}/gaussian_optimized_xyz'
sdf_file_path_input = f'{input_dir_path}/gaussian_optimized_mols.sdf'
radius_list = [1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6]
radius_for_stericmap = 0

1. **center_C_attached_O**

In [ ]:
center_atom = 'C'
attached_atom = 'O'
keyword = f'center_{center_atom}_attached_{attached_atom}'
dataset_file_path_input = f'{input_dir_path}/input_dataset_alcohol_oxidation.csv'
dataset_file_path_output = f'{fd}/morfeus_{keyword}.csv'

process_rows_for_morfeus(fd, dataset_file_path_input, xyz_dir_path_input, sdf_file_path_input, dataset_file_path_output, 
                            center_atom, attached_atom, keyword, radius_list, radius_for_stericmap, njobs=-1)
print('Finish')

2. **center_O_attached_C**

In [ ]:
center_atom = 'O'
attached_atom = 'C'
keyword = f'center_{center_atom}_attached_{attached_atom}'
dataset_file_path_input = f'{fd}/morfeus_center_C_attached_O.csv'
dataset_file_path_output = f'{fd}/morfeus_{keyword}.csv'

process_rows_for_morfeus(fd, dataset_file_path_input, xyz_dir_path_input, sdf_file_path_input, dataset_file_path_output, 
                            center_atom, attached_atom, keyword, radius_list, radius_for_stericmap, njobs=-1)
print('Finish')

3. **center_H(O)_attached_O**

In [ ]:
center_atom = 'H(O)'
attached_atom = 'O'
keyword = f'center_{center_atom}_attached_{attached_atom}'
dataset_file_path_input = f'{fd}/morfeus_center_O_attached_C.csv'
dataset_file_path_output = f'{fd}/morfeus_{keyword}.csv'

process_rows_for_morfeus(fd, dataset_file_path_input, xyz_dir_path_input, sdf_file_path_input, dataset_file_path_output, 
                            center_atom, attached_atom, keyword, radius_list, radius_for_stericmap, njobs=-1)
print('Finish')

4. **center_H(C)_attached_C**

In [ ]:
center_atom = 'H(C)'
attached_atom = 'C'
keyword = f'center_{center_atom}_attached_{attached_atom}'
dataset_file_path_input = f'{fd}/morfeus_center_H(O)_attached_O.csv'
dataset_file_path_output = f'{fd}/morfeus_{keyword}.csv'

process_rows_for_morfeus(fd, dataset_file_path_input, xyz_dir_path_input, sdf_file_path_input, dataset_file_path_output, 
                            center_atom, attached_atom, keyword, radius_list, radius_for_stericmap, njobs=-1)
print('Finish')

## 4. Generating Steric Descriptors with Module 2

Next, we used `morfeus_vbur_quadrant_octant_alcohol.py` (Module 2), which provides the function `process_rows_for_morfeus_qo`.  
This function computes quadrant- and octant-partitioned %Vbur, based on the following fixed atoms:
- Center: Carbon (C)
- Z-axis definition: Oxygen (O)
- XZ-plane definition: Hydrogen attached to that carbon (H(C))

Evaluation of the steric descriptors generated by Module 1 using the yield prediction model revealed that the %Vbur calculated with carbon as the center atom yielded the highest predictive accuracy. Accordingly, quadrant and octant analyses were conducted using these settings. Details of the yield prediction model are provided in `Tutorial 2`.

In [ ]:
from src.morfeus_vbur_quadrant_octant_alcohol import process_rows_for_morfeus_qo

keyword = f'centerC_zaxisO_xzplaneH(C)'
radius_for_stericmap = 2.5
dataset_file_path_input = f'{fd}/morfeus_center_H(C)_attached_C.csv'
dataset_file_path_output = f'{parent_dir}/data/output_data/morfeus_qo_{keyword}.csv'

process_rows_for_morfeus_qo(fd, dataset_file_path_input, xyz_dir_path_input, sdf_file_path_input, dataset_file_path_output, 
                            keyword, radius_list, radius_for_stericmap, njobs=-1)
print('Finish')

## 5. Generating Steric Profiles

To visualize the steric diversity of the alcohols in the dataset, steric profiles were generated by scanning the radius and plotting the buried volume. This approach enables the identification of the radius within which all substituents are accommodated, providing an overview of the molecular shape. To facilitate this identification, buried volume, rather than %Vbur, was used. Additionally, quadrant analysis was employed to visualize the steric environment on both the reactive site and the opposite side.

For visualization, the steric profiles were generated using the `process_rows_for_morfeus_profile` function in the `morfeus_steric_profile_alcohol.py` module.

In [ ]:
import numpy as np
from src.morfeus_steric_profile_alcohol import process_rows_for_morfeus_profile

keyword = f'centerC_zaxisO_xzplaneH(C)_sp'
radius_list = [round(r, 1) for r in np.arange(1.5, 15.1, 0.1)]
radius_for_stericmap = 10
dataset_file_path_input = f'{input_dir_path}/input_dataset_alcohol_oxidation.csv'

process_rows_for_morfeus_profile(fd, dataset_file_path_input, xyz_dir_path_input, sdf_file_path_input, 
                                 keyword, radius_list, radius_for_stericmap, njobs=-1)
print('Finish')